# PID Control & Motors

This week we'll be learning about controlling systems with feedback (including systems with electrical motors!). We're goign to start with servo motors since they're the simplest. Consider the following Arduino Code[1, simplified version Sparkfun Inventors Kit v4.0, Servo Project]:

    #include <Servo.h>   //include the servo library

    int potPosition;     //this variable will store the position of the potentiometer
    int servoPosition;   //the servo will move to this position

    Servo myservo;       //create a servo object

    void setup() {

      myservo.attach(9); //tell the servo object that its servo is plugged into pin 9

    }

    void loop() {

        //use analog read to measure the position of the potentiometer (0-1023)

        potPosition = analogRead(A0);        
      
        //convert the potentiometer number to a servo position from 20-160
        //Note: its best to avoid driving the little SIK servos all the 
        //way to 0 or 180 degrees it can cause the motor to jitter, which is bad for the servo.

        servoPosition = map(potPosition, 0,1023,20,160);

        myservo.write(servoPosition); //move the servo to the 10 degree position
    }

Build this program and attach a 0-5V potentiometer on A0 and the servo control line on pin 9. The servo-motor will also need power, which you can take from Vin and GND of the RedBoard.

Use the oscilloscope to study the behavior of the control line as you adjust the potentiometer and witness the motion of the servo. What changes about the pulses driving the control line as you adjust the voltage on pin A0?

There is a significant complexity buried in the "servo" object here (FYI you can find the servo object implementation under the Arduino Application 'libraries' folder). Let's dig in a bit deeper and implement this same functionality on the Cypress 5LP board.

We'll need to use an ADC (Analog to Digital Converter) and a PWM (Pulse Width Modulation) output.

Create a new Cypress 5LP project like this:

![Cypress 5LP Version](5LP-ServoControl.png)

The PWM component allows you to produce a variety of digital waveforms based on a clock input, a counter and a compare value. The idea is that the period of the waveform is an integer multiple of clock cycles and the waveform transitions when the counter reaches specific values. The counter starts each cycle with a specific value (period). It counts *down* on each clock cycle and performs the specific comparison (CMP Type) with the corrsponding "CMP Value". If the comparison is "true" the output goes high. If it's "false", the output goes low. Here's the default configuration dialog:

![Default PWM Configuration Dialog](PWM_Config.png)

So to configure this you need to consider: What period should the PWM output have? What range of pulse widths do we need to support? Give these some thought and try it! Check that you can reliably produce a waveform similar to those you measured on the RedBoard. If you get stuck, ask! You'll only need a single output for this project, but you'll probably need a 16 bit counter rather than the default 8 bit counter (why?).

We can adjust the CMP Value in software by calling:

![Write Compare](API_WriteCompare.png)

Also, don't forget to start the PWM component with "PWM_1_Start()".

We also need an ADC in this project to match the `A0` input from the RedBoard project. Drag an ADC_SAR into your design and set it up as a 10 bit ADC (to match the RedBoard). Note that unlike the RedBoard (Arduino) limit of 10000 samples/sec, the 5LP can sample up to a million samples per second at 12 bits of resolution! We don't need that speed for this project, but there are times when this kind of speed is very useful.

You must start the ADC with ADC_SAR_1_Start(). Also you can begin a conversion using:

![Start](ADCStart.png)

You can wait for a conversion to complete using:

![Ready](ADCIsEnd.png)

using the ADC_SAR_1_WAIT_FOR_RESULT flag. 

Once the result is ready, you can read it using: ADC_SAR_1_GetResult16:

![GetResult](ADC_GetResult.png)

[1] You can get the original code [From Sparkfun](https://learn.sparkfun.com/tutorials/sparkfun-inventors-kit-experiment-guide---v40/circuit-3a-servo-motors).

You may find it useful to incoporate the Arduino `map` function in your project as well:

    long map(long x, long in_min, long in_max, long out_min, long out_max)
    {
      return (x - in_min) * (out_max - out_min) / (in_max - in_min) + out_min;
    }

Next we'll add a bit of feedback and control the motors position using a PID control process.

A PID control system uses "Proportional", "Integral" and "Differential" feedback to control a system in such a way that it maintains a desired condition. We're going to use this idea to control a servo motor.

For your reference, here's a "high power" [deep dive on PID control](https://www.cds.caltech.edu/~murray/courses/cds101/fa02/caltech/astrom-ch6.pdf) while there are also more [gentle introductions available.](https://www.vexforum.com/index.php/attachment/59e1993a1d9e5_introduction_to_pid_controllers_ed2(1).pdf).

For today you can just grab some sources I cooked up (based loosely on the Arduino PID library):

[simplePID.h](simplePID.h)

[simplePID.c](simplePID.c)

Just copy these two to your Cypress project directory (right next to main.c) and add them to your project:

![Adding a file](AddExisting.png)

You'll need to add a timer to the circuit (used by the PID functions) like so:

![New Circuit](AddTimer.png)

This timer is used to get a "millis" function that behaves like the "millis" function on the Arduino. Makes sure the timer is driven by a 1kHz clock. And add the new "include" directive to main.c:

![New Include](NewInclude.png)

For the input voltage we'll be using a photodiode + amplifier called an [OPT-101](http://www.ti.com/lit/ds/symlink/opt101.pdf).

Use this as the input (rather than the power supply).

Your instructor will provide a variable transparency encoder that turns rotation of the motor into a variable light level on the photodiode. This can be used as a crude feedback mechanism to control the position of the motor. Your instructor will review the PID code with you. Please set up the circuit + code and demonstrate that the feedback attempts to keep the light level on the photodiode constant by turning the motor. This is easiest to see by monitoring the pulse width and the light level with your oscilloscope.
